In [ ]:
import numpy as np

In [ ]:
x = np.array([0, 0, -0.068, -0.137, -0.137, -0.137, -0.162])
y = np.array([0, 0, 0, 0, 0.2, 0.39, 0.463])
z = np.array([0.12, 0.32, 0.509, 0.696, 0.7, 0.7, 0.687])
roll = np.array([0, 0, 0.349, 1.569, -1.57, -1.694, -1.694])
pitch = np.array([0, -0.349, 0, -1.222, -0.348, -0.362, -0.326])
yaw = np.array([0, 0, -1.571, -3.14, 0, 0.372, 0.372])

axes = np.array([[0, 0, 1], [0, -1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, -1, 0], [0, 0, 1]])
origins = np.array([x, y, z]).T
angles_rpy = np.array([roll, pitch, yaw]).T

In [ ]:
def rotate(angles_rpy, vector):
    gamma = angles_rpy[0]
    beta = angles_rpy[1]
    alpha = angles_rpy[2]
    b = vector / np.linalg.norm(vector)
    
    sin = np.sin
    cos = np.cos
    
    a = np.array([[cos(alpha)*cos(beta), cos(alpha)*sin(beta)*sin(gamma) - sin(alpha)*cos(gamma), cos(alpha)*sin(beta)*cos(gamma) + sin(alpha)*sin(gamma)],
                 [sin(alpha)*cos(beta), sin(alpha)*sin(beta)*sin(gamma) + cos(alpha)*cos(gamma), sin(alpha)*sin(beta)*cos(gamma) - cos(alpha)*sin(gamma)],
                 [-sin(beta), cos(beta)*sin(gamma), cos(beta)*cos(gamma)]])
    
    c = a.dot(b)
    
    return c / np.linalg.norm(c)


def build_jacobian(origins, angles_rpy, axes, endeffector=origins[-1], offset=0):
    j = np.array([np.concatenate([np.cross(rotate(angle, axis), endeffector - origin), rotate(angle, axis)])
                  for origin, angle, axis in zip(origins[offset:], angles_rpy[offset:], axes[offset:])])
    
    return j.T

In [ ]:
jacobian = build_jacobian(origins, angles_rpy, axes)

np.set_printoptions(precision=3, linewidth=120)
print(jacobian)

In [ ]:
force = np.concatenate([rotate(angles_rpy[-1], [0, 0, 1]), [0, 0, 0]])
torque = jacobian.T.dot(force)

print('Force vector: {}'.format(force))
print('Joint torques: {}'.format(torque))

In [ ]:
# Hand to base transformation
ee = np.array([rotate(angles_rpy[-1], axis) for axis in ([1, 0, 0], [0, 1, 0], [0, 0, 1])])
htb = np.zeros((6, 6))
htb[0:3,0:3] = htb[3:,3:] = np.linalg.inv(ee)
jacobian_htb = np.matmul(htb, jacobian)
torque = np.matmul(jacobian_htb.T, np.array([0, 0, 1, 0, 0, 0]))

print('Joint torques: {}'.format(torque))

In [ ]:
joints_deg = np.array([60.98, 119.42, -81.44, 80.38, 50.09, -151.34, -103.95])
joints_rad = np.pi / 180 * joints_deg

print('Joint angles: {}'.format(joints_rad))